<a href="https://colab.research.google.com/github/arthur-siqueira/interpolacao-sistemas-lineares/blob/main/interpolacao_sistemas_lineares.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt


def interpolacao_polinomial(x_pontos, y_pontos):
  """
  Calcula os coeficientes de um polinomio interpolador.
  A função gera e resolve o sistema linear Ac = y, onde A é a matriz de Vandermonde.

  Args:
    x_pontos (np.array): Coordenadas x dos pontos.
    y_pontos (np.array): Coordenadas y dos pontos.

  Returns:
    np.array: Vetor de coeficientes do polinômio [c0, c1, ..., cn-1]
  """

  n = len(x_pontos)
  # Gera a matriz de Vandermonde A
  A = np.zeros((n, n))
  for i in range(n):
    for j in range(n):
      A[i, j] = x_pontos[i]**j

  # Resolve um sistema linear Ac = y para encontrar os coeficientes c
  c = np.linalg.solve(A, y_pontos)

  return c